In [1]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.cluster import KMeans
from msresist.comp_estimator import ComHyperPar, MyOwnKMEANS
from msresist.plsr import ClusterAverages, GridSearch_CV, MeasuredVsPredicted_LOOCVplot, Q2Y_across_components, R2Y_across_components
from msresist.pre_processing import preprocessing, MergeDfbyMean, FoldChangeFilter
from msresist.sequence_analysis import GeneratingKinaseMotifs
import scipy as sp, numpy as np, pandas as pd, math
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import seaborn as sns; sns.set(color_codes=True)
from FileExporter import create_download_link
from Bio import SeqIO
import warnings
warnings.simplefilter("ignore")

# Importing and pre-processing data

### Phosphorylation levels of peptides across conditions | X Matrix where peptides = vars and conditions = obs

Preprocessing: Each peptide is mean-centered across conditions, merged, converted to fold-change to the control, and log2-transformed. Data can be merged by raw sequences or mapped motifs.

In [2]:
np.power(2, -0.015025)

0.9896395071207668

In [3]:
A_r = pd.read_csv('./msresist/data/Raw/20180817_JG_AM_TMT10plex_R1_psms_raw.csv', header=0)
B_r = pd.read_csv('./msresist/data/Raw/20190214_JG_AM_PC9_AXL_TMT10_AC28_R2_PSMs_raw.csv', header=0)
C_r = pd.read_csv('./msresist/data/Raw/CombinedBR3_TR1&2_raw.csv', header=0)

treatments = A_r.columns[2:]
header = A_r.columns

ABC_mc = preprocessing(A_r, B_r, C_r, motifs=False, Vfilter=True, FCfilter=True, log2T=True)

,peptide-phosphosite,Master Protein Descriptions,PC9,Erl,R428,Erl/R428,Erl+HGF,Erl+FGF,Erl+IGF,KO Erl,KO R428,KO Erl/R428
0,aAVPSGASTGIyEALELR-1,Alpha-enolase OS=Homo sapiens GN=ENO1 PE=1 SV=...,0.380463,-0.252746,-0.099739,-0.081470,0.041829,-0.197131,0.266632,-0.079869,0.138428,-0.116398
1,aAVPSGASTGIyEALELRDNDk-1,Alpha-enolase OS=Homo sapiens GN=ENO1 PE=1 SV=...,0.198171,-0.082829,-0.242864,-0.265561,-0.144468,0.138360,-0.005705,0.138107,0.210536,0.056253
2,aAVPSGAStGIYEALELRDNDkTR-1,Alpha-enolase OS=Homo sapiens GN=ENO1 PE=1 SV=...,0.176992,0.059421,-0.222378,-0.075176,-0.188601,0.395127,-0.390902,0.137421,0.253186,-0.145090
187,sFIkDyPVVSIEDPFDQDDWGAWQk-1,Alpha-enolase OS=Homo sapiens GN=ENO1 PE=1 SV=...,0.359546,0.282541,0.203408,0.225499,0.115526,0.388391,0.042509,-0.585832,-0.520109,-0.511479
0,aAVPSGASTGIyEALELR-1,Alpha-enolase OS=Homo sapiens OX=9606 GN=ENO1 ...,0.423461,0.146240,-0.010170,-0.109502,0.111169,0.099958,0.109984,-0.216915,-0.119366,-0.434859
3,aAVPSGASTGIyEALELR-1,Alpha-enolase OS=Homo sapiens OX=9606 GN=ENO1 ...,0.325665,0.482128,0.012906,0.101149,0.146988,0.165575,-0.071251,-0.230363,-0.393011,-0.539785
4,aAVPSGASTGIyEALELRDNDk-1,Alpha-enolase OS=Homo sapiens OX=9606 GN=ENO1 ...,0.356259,0.446898,-0.101581,0.030837,0.052381,0.140936,-0.185116,-0.422506,-0.024999,-0.293108
525,sFIkDyPVVSIEDPFDQDDWGAWQk-1,Alpha-enolase OS=Homo sapiens OX=9606 GN=ENO1 ...,-0.156800,0.206946,0.201335,0.202587,0.071061,0.091181,0.284809,-0.596235,-0.270565,-0.034319
786,yNQLLR-1,Alpha-enolase OS=Homo sapiens OX=9606 GN=ENO1 ...,0.329903,0.178406,-0.332534,-0.360492,0.130111,0.153811,-0.201053,-0.192573,0.152237,0.142184


,peptide-phosphosite,Master Protein Descriptions,PC9,Erl,R428,Erl/R428,Erl+HGF,Erl+FGF,Erl+IGF,KO Erl,KO R428,KO Erl/R428
407,yNQLLR,Alpha-enolase,0.0,-0.151497,-0.662437,-0.690395,-0.199792,-0.176092,-0.530956,-0.522476,-0.177666,-0.187719
409,AVPSGAStGIYEALELRDNDkTR,Alpha-enolase,0.0,-0.117571,-0.399370,-0.252168,-0.365592,0.218135,-0.567894,-0.039571,0.076195,-0.322081
26,AVPSGASTGIyEALELRDNDk,Alpha-enolase,0.0,-0.083246,-0.449413,-0.392920,-0.323128,-0.135473,-0.362779,-0.375116,-0.171063,-0.373446
27,sFIkDyPVVSIEDPFDQDDWGAWQk,Alpha-enolase,0.0,0.160137,0.123793,0.133664,0.011128,0.142570,0.111589,-0.670340,-0.446471,-0.290388
4,AVPSGASTGIyEALELR,Alpha-enolase,0.0,-0.215094,-0.407125,-0.400347,-0.274818,-0.343625,-0.269796,-0.550369,-0.487601,-0.729677


SystemExit: 

In [ ]:
ABC_mc.iloc[:50, :]

In [ ]:
raise SystemExit

In [ ]:
Xf_data = ABC_mc.iloc[:,2:].T
Xf_protnames = list(ABC_mc.iloc[:,1])
Xf_seqs = list(ABC_mc.iloc[:,0])

### Phenotypes

Cell Viability:

In [ ]:
Y_cv = pd.read_csv('./msresist/data/Phenotypic_data/viability.csv')
Y_cv = Y_cv.iloc[:10, 2]

## Phosphopeptides Heat Map

In [ ]:
# g = sns.clustermap(Xf_data.iloc[:, 2:].copy().T, method = "average", robust = True)
# g

In [ ]:
# Xf.iloc[:, 2:] = np.log2(Xf.iloc[:, 2:].values)
# Xf_data = np.log2(Xf_data.values)

In [ ]:
# p = g.dendrogram_row.reordered_ind

Recovering heat map peptides 

In [ ]:
# p = g.dendrogram_row.reordered_ind

# hm_prots = []
# for i in p:
#     hm_prots.append(Xf.iloc[i,:])
# hm_prots = pd.DataFrame(hm_prots)

## Cell Viability Measurements

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(5,3))
plt.bar(np.linspace(1,10,10)-1, Y_cv,width=0.3,align='center', color=['black', 'darkred', 'darkred', 'darkred', 'darkgreen', 'darkgreen', 'darkgreen', 'darkblue', 'darkblue', 'darkblue'])
plt.xticks(np.arange(10), (treatments), rotation=70)
plt.title("Cell Viability")
plt.ylabel("Viability (fold change)")
# plt.savefig("CellViability.pdf")
plt.show()

# GridSearchCV

Simultaneous hyperparameter tuning of number of clusters and components across entire model pipeline

In [ ]:
import warnings
warnings.simplefilter("ignore")

CVresults_max, CVresults_min, best_params = ComHyperPar(Xf_data, Y_cv, Xf_protnames, Xf_seqs)

In [ ]:
print(best_params)
display(CVresults_min.iloc[:20,:])
# create_download_link(CVresults_min, "GridSearch_ClustVsComp")

In [ ]:
# MSE = np.abs(CVresults_min.iloc[:20, 2])
#range_ = np.linspace(1,MSE.shape[0],MSE.shape[0])

#fig, axs = plt.subplots(1,1,figsize=(7,5))
#plt.setp(axs, xticks=range_)
#plt.bar(range_,MSE,width=0.3,align='center', color = "green")
# plt.savefig("GridSearch_Cl&Comp.pdf")
#plt.title("GridSearch Cell Viability")
#plt.xlabel("Number of Components")
#plt.ylabel("MSE")
#plt.show()

# R2Y / Q2Y

Percentages of variance explained (R2Y) and predicted (Q2Y) across numbers of components

In [ ]:
import warnings
warnings.simplefilter("ignore")

maxComp = 10
Q2Y = Q2Y_across_components(Xf_data, Y_cv, maxComp+1)
R2Y = R2Y_across_components(Xf_data, Y_cv, maxComp+1)

First using the entire data set

In [ ]:
def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n

def disp_values(x,spac1,spac2,dec):
    for l, m in enumerate(x):
        plt.text(l+spac1,m+spac2,str(truncate(m,dec)))    #change to axs[ax].

range_ = np.linspace(1,maxComp,maxComp)

fig, axs = plt.subplots(1,1,figsize=(9,7))
plt.setp(axs, xticks=range_)
plt.bar(range_+0.15,Q2Y,width=0.3,align='center',label='Q2Y', color = "darkred")
plt.bar(range_-0.15,R2Y,width=0.3,align='center',label='R2Y', color = "black")
plt.title("R2Y/Q2Y Cell Viability")
plt.xlabel("Number of Components")
plt.legend(loc=4)
disp_values(Q2Y,1.03,0.01,3)
disp_values(R2Y,0.6,0.01,3)

Then after k-means clustering setting the number of clusters to 5

In [ ]:
kmeans = MyOwnKMEANS(5, Xf_protnames, Xf_seqs).fit(Xf_data, Y=None)
centers = kmeans.transform(Xf_data)
clustermembers = kmeans.ClusterMembers(Xf_data)

In [ ]:
maxComp = centers.shape[1]
Q2Y = Q2Y_across_components(centers, Y_cv, maxComp)
R2Y = R2Y_across_components(centers, Y_cv, maxComp)

In [ ]:
def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n

def disp_values(x,spac1,spac2,dec):
    for l, m in enumerate(x):
        plt.text(l+spac1,m+spac2,str(truncate(m,dec)))    #change to axs[ax].

range_ = np.linspace(1,maxComp-1,maxComp-1)

fig, axs = plt.subplots(1,1,figsize=(5,3))
plt.setp(axs, xticks=range_)
plt.bar(range_+0.15,Q2Y,width=0.3,align='center',label='Q2Y', color = "darkblue")
plt.bar(range_-0.15,R2Y,width=0.3,align='center',label='R2Y', color = "black")
plt.title("R2Y/Q2Y Cell Viability")
plt.xlabel("Number of Components")
plt.legend(loc=4)
disp_values(Q2Y,1.03,0.02,3)
disp_values(R2Y,0.74,0.01,3)
# plt.savefig("R2YQ2Y_BR1_5Cl.pdf")

# Building chained k-means and PLSR model pipeline

## Measured Vs Predicted

We've learnt during cross-validation that by setting the number of components to 2 and the number of clusters to 5 the model should perform reasonably well at predicting the viability response across conditions. To corroborate that we plot measured vs predicted below. We also show that the model performs better after clustering than using the entire data set, or even the filtered data set. 

In [ ]:
ncl, ncomp = 5, 2
estimators = [('kmeans', MyOwnKMEANS(ncl, Xf_protnames, Xf_seqs)), ('plsr', PLSRegression(ncomp))]
pipe = Pipeline(estimators)

In [ ]:
X_scores, Y_scores = pipe.fit_transform(Xf_data, Y_cv)
PC1_scores, PC2_scores = X_scores[:, 0], X_scores[:, 1]
PC1_xload, PC2_xload = pipe.named_steps.plsr.x_loadings_[:, 0], pipe.named_steps.plsr.x_loadings_[:, 1]
PC1_yload, PC2_yload = pipe.named_steps.plsr.y_loadings_[:, 0], pipe.named_steps.plsr.y_loadings_[:, 1]

### Cross-Validation Strategy 1 - Pipeline:

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,5))
MeasuredVsPredicted_LOOCVplot(Xf_data,Y_cv, pipe, fig, "none", axs)
plt.title("Correlation Measured vs Predicted - CV across pipeline")
plt.xlabel("Measured Cell Viability")
plt.ylabel("Predicted Cell Viability")
#textstr = '\n'.join(['R: 0.79', 'p-value: 0.006'])
#props = dict(boxstyle = 'square', facecolor = "white", alpha = 0.5)
#plt.text(0.5, 14.7, textstr, bbox=props)
# plt.savefig('Measured_Predict_5cl2co_pipe.pdf')
plt.show()

### Cross-Validation Strategy 2 - Fixed Clusters:

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,5))
MeasuredVsPredicted_LOOCVplot(Xf_data,Y_cv, PLSRegression(ncomp), fig, "none", axs)
plt.title("Correlation Measured vs Predicted - Fixed Clustered Df")
plt.xlabel("Measured Cell Viability")
plt.ylabel("Predicted Cell Viability")
textstr = '\n'.join(['R: 0.79', 'p-value: 0.006'])
props = dict(boxstyle = 'square', facecolor = "white", alpha = 0.5)
plt.text(0.1, 10.5, textstr, bbox=props)
# plt.savefig('Measured_Predict_5cl2co_pipe.pdf')
plt.show()

### Scores and Loadings Plot

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(12,6))
# colors_ = cm.rainbow(np.linspace(0, 1, 5))
colors_ = ["blue", "orange", "green", "red", "purple"]


axs[0].scatter(PC1_scores,PC2_scores)
for j, txt in enumerate(treatments):
    axs[0].annotate(txt, (PC1_scores[j], PC2_scores[j]))
axs[0].set_title('PLSR Model Scores')
axs[0].set_xlabel('PC1')
axs[0].set_ylabel('PC2')
axs[0].axhline(y=0, color='0.25', linestyle='--')
axs[0].axvline(x=0, color='0.25', linestyle='--')
axs[0].set_xlim([-5, 5])
axs[0].set_ylim([-1.75, 1.75])

for i, txt in enumerate(["1", "2", "3", "4", "5"]):
    axs[1].annotate(txt, (PC1_xload[i], PC2_xload[i]))
axs[1].scatter(PC1_xload, PC2_xload, c=np.arange(ncl), cmap=colors.ListedColormap(colors_))
axs[1].scatter(PC1_yload, PC2_yload, color='#000000', marker='D', label='Cell Viability')
axs[1].legend(loc=4)
axs[1].set_title('PLSR Model Loadings (Averaged Clusters)')
axs[1].set_xlabel('PC1')
axs[1].set_ylabel('PC2')
axs[1].axhline(y=0, color='0.25', linestyle='--')
axs[1].axvline(x=0, color='0.25', linestyle='--')
axs[1].set_xlim([-0.65, 0.65])
axs[1].set_ylim([-1.1, 1.1])
# plt.savefig('scores_loadings.pdf')
plt.show()

In [ ]:
ClusterMembers = pipe.named_steps.kmeans.ClusterMembers(Xf_data)
count = sum(len(v) for v in ClusterMembers.values())
CM_df = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in ClusterMembers.items() ]))
create_download_link(CM_df, 'fsdf')

### Cluster Averages across Conditions

In [ ]:
Centers = pipe.named_steps.kmeans.transform(Xf_data)
memb = pipe.named_steps.kmeans.ClusterMembers(Xf_data)


fig = plt.figure(figsize=(6,4))

colors_ = ["blue", "orange", "green", "red", "purple"]

centersT = np.transpose(Centers)
for i in range(centersT.shape[0]):
    plt.plot(centersT[i,:], label = "cluster "+str(i+1), color = colors_[i])
    
plt.xticks(np.arange(centersT.shape[1]), (treatments), rotation=70)
# plt.legend(loc=2, prop={'size':8})
# plt.savefig("ClAvgAcrossCond_BR1.pdf")
plt.show()
display(pd.DataFrame(centersT))